<a href="https://colab.research.google.com/github/uditdayani/footyval0/blob/main/footyval0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

s1 = pd.read_csv('/content/pl_19-20.csv')
s2 = pd.read_csv('/content/pl_20-21.csv')
s3 = pd.read_csv('/content/pl_21-22.csv')
s4 = pd.read_csv('/content/pl_22-23.csv')
s5 = pd.read_csv('/content/pl_23-24.csv')
market_value_data = pd.read_csv('/content/players.csv')

print(s1.head())
print(s2.head())
print(s3.head())
print(s4.head())
print(market_value_data.head())


   Unnamed: 0                Name    Position  Appearances  Clean sheets  \
0           0         Max Aarons     Defender           36           5.0   
1           1  Abdul Rahman Baba     Defender            0           0.0   
2           2      Tammy Abraham      Forward           34           NaN   
3           3          Che Adams      Forward           30           NaN   
4           4    Dennis Adeniran   Midfielder            0           NaN   

   Goals conceded  Tackles Tackle success %  Last man tackles  Blocked shots  \
0            71.0     48.0              63%               0.0            6.0   
1             0.0      0.0               0%               0.0            0.0   
2             NaN      8.0              NaN               NaN           14.0   
3             NaN      6.0              NaN               NaN           14.0   
4             NaN      0.0               0%               NaN            0.0   

   ...  Shooting accuracy %  Big chances missed  Saves  Penalt

In [2]:
def clean_names(df, name_col='Name'):
    df[name_col] = df[name_col].str.strip().str.lower()
    return df

# Clean player names
s1 = clean_names(s1)
s2 = clean_names(s2)
s3 = clean_names(s3)
s4 = clean_names(s4)
s5 = clean_names(s5)
market_value_data = clean_names(market_value_data, name_col='Name')

# Add PL season identifier
s1['Season'] = '2019-20'
s2['Season'] = '2020-21'
s3['Season'] = '2021-22'
s4['Season'] = '2022-23'
s5['Season'] = '2023-24'

# Concatenate performance data across seasons
combined_data = pd.concat([s1, s2, s3, s4, s5], ignore_index=True)

# Merge with the market value data
merged_data = pd.merge(combined_data, market_value_data[['Name', 'market_value_in_eur']],
                       on='Name', how='left')

print(merged_data.head())



merged_data=merged_data.dropna(subset=['market_value_in_eur'])

def convert_percent_to_numeric(df, cols):
    for col in cols:
        # Remove '%' and convert to numeric
        df[col] = df[col].str.replace('%', '').str.replace(',', '').astype(float) / 100
    return df

percent_columns = ['Shooting accuracy %', 'Cross accuracy %', 'Tackle success %']
merged_data = convert_percent_to_numeric(merged_data, percent_columns)



# Check for missing values in each column
missing_values = merged_data.isnull().sum()

print(merged_data['market_value_in_eur'].isnull().sum())

print(missing_values)

   Unnamed: 0               Name    Position  Appearances  Clean sheets  \
0         0.0         max aarons    Defender           36           5.0   
1         1.0  abdul rahman baba    Defender            0           0.0   
2         2.0      tammy abraham     Forward           34           NaN   
3         3.0          che adams     Forward           30           NaN   
4         4.0    dennis adeniran  Midfielder            0           NaN   

   Goals conceded  Tackles Tackle success %  Last man tackles  Blocked shots  \
0            71.0     48.0              63%               0.0            6.0   
1             0.0      0.0               0%               0.0            0.0   
2             NaN      8.0              NaN               NaN           14.0   
3             NaN      6.0              NaN               NaN           14.0   
4             NaN      0.0               0%               NaN            0.0   

   ...  High Claims  Catches  Sweeper clearances  Throw outs  Goal K

In [3]:
merged_data[['Punches', 'High Claims', 'Catches', 'Sweeper clearances','Goal Kicks','Goals Conceded','Big Chances Created',
'Saves','Shooting accuracy %','Shots on target','Shots','Freekicks scored','Penalties scored','Goals per match',
'Hit woodwork','Goals with left foot','Goals with right foot','Headed goals','Offsides','Accurate long balls',
'Through balls','Cross accuracy %','Crosses','Big chances created','Errors leading to goal','Own goals',
'Aerial battles lost','Aerial battles won','Successful 50/50s','Duels won','Duels lost','Clearances off line','Recoveries',
'Headed Clearance','Clearances','Interceptions','Blocked shots','Tackle success %','Last man tackles','Goals conceded',
'Tackles','Clean sheets','Penalties Saved']] = merged_data[['Punches', 'High Claims', 'Catches', 'Sweeper clearances','Goal Kicks','Goals Conceded','Big Chances Created',
'Saves','Shooting accuracy %','Shots on target','Shots','Freekicks scored','Penalties scored','Goals per match',
'Hit woodwork','Goals with left foot','Goals with right foot','Headed goals','Offsides','Accurate long balls',
'Through balls','Cross accuracy %','Crosses','Big chances created','Errors leading to goal','Own goals',
'Aerial battles lost','Aerial battles won','Successful 50/50s','Duels won','Duels lost','Clearances off line','Recoveries',
'Headed Clearance','Clearances','Interceptions','Blocked shots','Tackle success %','Last man tackles','Goals conceded',
'Tackles','Clean sheets','Penalties Saved']].fillna(0)


In [4]:
merged_data=merged_data.drop(columns=['Unnamed: 0', 'Big chances missed', 'Throw outs', 'Penalties Saved'])

missing_values = merged_data.isnull().sum()

print(missing_values)

Name                         0
Position                     0
Appearances                  0
Clean sheets                 0
Goals conceded               0
Tackles                      0
Tackle success %             0
Last man tackles             0
Blocked shots                0
Interceptions                0
Clearances                   0
Headed Clearance             0
Clearances off line          0
Recoveries                   0
Duels won                    0
Duels lost                   0
Successful 50/50s            0
Aerial battles won           0
Aerial battles lost          0
Own goals                    0
Errors leading to goal       0
Assists                      0
Passes                       0
Passes per match             0
Big chances created          0
Crosses                      0
Cross accuracy %             0
Through balls                0
Accurate long balls          0
Yellow cards                 0
Red cards                    0
Fouls                        0
Offsides

In [5]:
# Aggregate statistics across all seasons for each player
aggregated_data = merged_data.groupby('Name').agg({
    'Appearances': 'sum',
    'Goals': 'sum',
    'Assists': 'sum',
    'Tackles': 'sum',
    'Interceptions': 'sum',
    'Punches': 'sum',
    'High Claims': 'sum',
    'Catches': 'sum',
    'Sweeper clearances': 'sum',
    'Goal Kicks': 'sum',
    'Goals Conceded': 'sum',
    'Big Chances Created': 'sum',
    'Saves': 'sum',
    'Shooting accuracy %': 'mean',
    'Shots on target': 'sum',
    'Shots': 'sum',
    'Freekicks scored': 'sum',
    'Penalties scored': 'sum',
    'Goals per match': 'mean',
    'Hit woodwork': 'sum',
    'Goals with left foot': 'sum',
    'Goals with right foot': 'sum',
    'Headed goals': 'sum',
    'Offsides': 'sum',
    'Accurate long balls': 'sum',
    'Through balls': 'sum',
    'Cross accuracy %': 'mean',
    'Crosses': 'sum',
    'Big chances created': 'sum',
    'Errors leading to goal': 'sum',
    'Own goals': 'sum',
    'Aerial battles lost': 'sum',
    'Aerial battles won': 'sum',
    'Successful 50/50s': 'sum',
    'Duels won': 'sum',
    'Duels lost': 'sum',
    'Clearances off line': 'sum',
    'Recoveries': 'sum',
    'Headed Clearance': 'sum',
    'Clearances': 'sum',
    'Interceptions': 'sum',
    'Blocked shots': 'sum',
    'Tackle success %': 'mean',
    'Last man tackles': 'sum',
    'Goals conceded': 'sum',
    'Tackles': 'sum',
    'Clean sheets': 'sum',
    'market_value_in_eur': 'mean',
    'Season': 'first',
    'Position': 'first'
}).reset_index()

# Fill missing values with 0 for aggregated performance metrics
aggregated_data.fillna(0, inplace=True)

# Check for missing values
print(aggregated_data[['Appearances', 'Goals', 'Assists', 'Tackles', 'Interceptions']].isnull().sum())


Appearances      0
Goals            0
Assists          0
Tackles          0
Interceptions    0
dtype: int64


In [6]:
# Replace zero appearances with a small number to avoid division by zero
aggregated_data['Appearances'] = aggregated_data['Appearances'].replace(0, 1)

# Create per 90 minutes metrics for aggregated data
aggregated_data['goals_per_90'] = aggregated_data['Goals'] / aggregated_data['Appearances']
aggregated_data['assists_per_90'] = aggregated_data['Assists'] / aggregated_data['Appearances']
aggregated_data['tackles_per_90'] = aggregated_data['Tackles'] / aggregated_data['Appearances']
aggregated_data['interceptions_per_90'] = aggregated_data['Interceptions'] / aggregated_data['Appearances']
aggregated_data['blocked_shots_per_90'] = aggregated_data['Blocked shots'] / aggregated_data['Appearances']
aggregated_data['last_man_tackles_per_90'] = aggregated_data['Last man tackles'] / aggregated_data['Appearances']
aggregated_data['accurate_long_balls_per_90'] = aggregated_data['Accurate long balls'] / aggregated_data['Appearances']
aggregated_data['through_balls_per_90'] = aggregated_data['Through balls'] / aggregated_data['Appearances']
aggregated_data['big_chances_created_per_90'] = aggregated_data['Big chances created'] / aggregated_data['Appearances']
aggregated_data['clearances_per_90'] = aggregated_data['Clearances'] / aggregated_data['Appearances']
aggregated_data['clearances_off_line_per_90'] = aggregated_data['Clearances off line'] / aggregated_data['Appearances']


# Offensive score (weighted)
aggregated_data['offensive_score'] = (aggregated_data['goals_per_90'] * 0.5) + (aggregated_data['assists_per_90'] * 0.3) + (aggregated_data['Accurate long balls'] * 0.06) + (aggregated_data['Through balls'] * 0.06) + (aggregated_data['Big chances created'] * 0.08)

# Defensive score (weighted)
aggregated_data['defensive_score'] = (aggregated_data['tackles_per_90'] * 0.3) + (aggregated_data['interceptions_per_90'] * 0.5) + (aggregated_data['Blocked shots']*0.05) + (aggregated_data['Clearances']*0.05) + (aggregated_data['Clearances off line']*0.05) + (aggregated_data['Last man tackles']*0.05)

# Goalkeeper score
aggregated_data['goalkeeper_score'] = (
    (aggregated_data['Saves'] * 0.4) +
    (aggregated_data['Clean sheets'] * 0.3) +
    (aggregated_data['Punches'] * 0.1) +
    (aggregated_data['High Claims'] * 0.2)
)

# Overall weighted score based on position
aggregated_data['overall_score'] = aggregated_data.apply(
    lambda x: x['offensive_score'] if 'Forward' in x['Position']
    else (x['defensive_score'] if 'Defender' in x['Position']
    else (x['goalkeeper_score'] if 'Goalkeeper' in x['Position']
    else (x['offensive_score'] + x['defensive_score']) / 2)), axis=1
)

# Value-to-Performance Ratio
aggregated_data['value_to_performance'] = aggregated_data['market_value_in_eur'] / aggregated_data['overall_score']

print(aggregated_data[['Name', 'Position', 'overall_score', 'market_value_in_eur', 'value_to_performance', 'Season']].head())

# Check for missing values
print(aggregated_data.isnull().sum())



                Name  Position  overall_score  market_value_in_eur  \
0     aaron connolly   Forward       0.148889            3500000.0   
1    aaron cresswell  Defender       4.280952             900000.0   
2     aaron donnelly  Defender       0.000000             500000.0   
3       aaron hickey  Defender       2.598571           28000000.0   
4  aaron keto-diyawa  Defender       0.000000             200000.0   

   value_to_performance   Season  
0          2.350746e+07  2019-20  
1          2.102336e+05  2019-20  
2                   inf  2022-23  
3          1.077515e+07  2022-23  
4                   inf  2023-24  
Name                    0
Appearances             0
Goals                   0
Assists                 0
Tackles                 0
                       ..
offensive_score         0
defensive_score         0
goalkeeper_score        0
overall_score           0
value_to_performance    0
Length: 65, dtype: int64


In [7]:

num_rows = aggregated_data.shape[0]
print(f"Number of rows in aggregated_data: {num_rows}")



Number of rows in aggregated_data: 1762
Rows with missing original data:
Empty DataFrame
Columns: [Name, Appearances, Goals, Assists, Tackles, Interceptions, Punches, High Claims, Catches, Sweeper clearances, Goal Kicks, Goals Conceded, Big Chances Created, Saves, Shooting accuracy %, Shots on target, Shots, Freekicks scored, Penalties scored, Goals per match, Hit woodwork, Goals with left foot, Goals with right foot, Headed goals, Offsides, Accurate long balls, Through balls, Cross accuracy %, Crosses, Big chances created, Errors leading to goal, Own goals, Aerial battles lost, Aerial battles won, Successful 50/50s, Duels won, Duels lost, Clearances off line, Recoveries, Headed Clearance, Clearances, Blocked shots, Tackle success %, Last man tackles, Goals conceded, Clean sheets, market_value_in_eur, Season, Position, goals_per_90, assists_per_90, tackles_per_90, interceptions_per_90, blocked_shots_per_90, last_man_tackles_per_90, accurate_long_balls_per_90, through_balls_per_90, big_

In [9]:
import numpy as np
import pandas as pd

# Calculate the median value_to_performance
aggregated_data['value_to_performance'] = pd.to_numeric(aggregated_data['value_to_performance'], errors='coerce')
aggregated_data['value_to_performance'].replace([np.inf, -np.inf], np.nan, inplace=True)

median_value_to_performance = aggregated_data['value_to_performance'].median()

# Set thresholds for undervalued and overvalued players
undervalued_threshold = 0.5 * median_value_to_performance
overvalued_threshold = 1.5 * median_value_to_performance

aggregated_data['overall_score'] = aggregated_data['overall_score'].replace(0, 0.001)

# Classify players
def classify_valuation(row):
    if pd.isna(row['value_to_performance']):
        return 'Not enough data'  # Or handle as needed
    elif row['value_to_performance'] == np.inf:
        return 'Overvalued'
    elif row['value_to_performance'] < undervalued_threshold:
        return 'Undervalued'
    elif row['value_to_performance'] > overvalued_threshold:
        return 'Overvalued'
    else:
        return 'Fairly Valued'

# Apply the classification function to the dataframe
aggregated_data['valuation_status'] = aggregated_data.apply(classify_valuation, axis=1)

# Some results
print(aggregated_data[['Name', 'Position', 'market_value_in_eur', 'overall_score', 'value_to_performance', 'valuation_status', 'Season']].tail())

# Count of players classified in each category
print(aggregated_data['valuation_status'].value_counts())


                  Name    Position  market_value_in_eur  overall_score  \
1757       álex moreno    Defender           13000000.0         3.9575   
1758  álvaro fernández  Goalkeeper            1300000.0        43.8000   
1759     álvaro morata     Forward           16000000.0         0.0010   
1760            ângelo     Forward           15000000.0         0.0010   
1761      ørjan nyland  Goalkeeper            1400000.0         7.0000   

      value_to_performance valuation_status   Season  
1757          3.284902e+06    Fairly Valued  2022-23  
1758          2.968037e+04      Undervalued  2021-22  
1759                   NaN  Not enough data  2019-20  
1760                   NaN  Not enough data  2022-23  
1761          2.000000e+05      Undervalued  2019-20  
valuation_status
Not enough data    795
Overvalued         405
Undervalued        354
Fairly Valued      208
Name: count, dtype: int64


In [10]:
print(aggregated_data[aggregated_data['Name'] == "william saliba"])


                Name  Appearances  Goals  Assists  Tackles  Interceptions  \
1717  william saliba           65      4        2     75.0           48.0   

      Punches  High Claims  Catches  Sweeper clearances  ...  \
1717      0.0          0.0      0.0                 0.0  ...   

      through_balls_per_90  big_chances_created_per_90  clearances_per_90  \
1717              0.046154                         0.0           2.461538   

      clearances_off_line_per_90  offensive_score  defensive_score  \
1717                         0.0             8.14         9.115385   

      goalkeeper_score  overall_score  value_to_performance  valuation_status  
1717               8.7       9.115385          8.776371e+06        Overvalued  

[1 rows x 66 columns]


In [11]:
# Rank players based on their overall score
aggregated_data['rank_overall'] = aggregated_data['overall_score'].rank(ascending=False, method='min')

# Rank players within their playing position
aggregated_data['rank_by_position'] = aggregated_data.groupby('Position')['overall_score'].rank(ascending=False, method='min')

# Show ranked results
print(aggregated_data[['Name', 'Position', 'overall_score', 'market_value_in_eur', 'rank_overall', 'rank_by_position', 'Season']].sort_values(by='rank_overall').head(20))

# Display top 5 players in each position
top_5_by_position = aggregated_data.groupby('Position').apply(lambda x: x.nsmallest(10, 'rank_by_position')).reset_index(drop=True)
print(top_5_by_position[['Name', 'Position', 'overall_score', 'market_value_in_eur', 'rank_by_position', 'Season']])


                   Name    Position  overall_score  market_value_in_eur  \
474   emiliano martínez  Goalkeeper          473.2         1.600000e+07   
452             ederson  Goalkeeper          324.2         1.785000e+07   
1284               neto  Goalkeeper          310.8         9.333333e+05   
840     jordan pickford  Goalkeeper          260.8         2.200000e+07   
1287          nick pope  Goalkeeper          248.5         1.600000e+07   
195          bernd leno  Goalkeeper          228.4         1.300000e+07   
7        aaron ramsdale  Goalkeeper          222.7         2.500000e+07   
1053   lukasz fabianski  Goalkeeper          204.8         7.000000e+05   
1677     vicente guaita  Goalkeeper          186.8         8.000000e+05   
631         hugo lloris  Goalkeeper          180.5         1.200000e+06   
871             josé sá  Goalkeeper          180.1         1.400000e+07   
377        david de gea  Goalkeeper          178.4         5.000000e+06   
644       illan meslier  